# Sprint 3 · Webinar 10 — Clase Práctica de SQL (RunSQL)

**Enfoque:** profundización a partir del cuaderno teórico: KPIs financieros, validación de resultados, reporte y tendencias.

**Duración sugerida:** 120–150 min  
**Autor:** Tripleten MT

> Copia y pega cada bloque SQL en **RunSQL** (dialecto PostgreSQL).

<div style="text-align: center">
    <img src="https://raw.githubusercontent.com/ljpiere/tpdata_python/main/images/w1s1_2.png" width="400">
</div>

## Objetivos
1. Reforzar KPIs (revenue, cost, gross_profit, margin%) y **ROI**.
2. Aplicar **CTEs**, **CASE**, **subconsultas** y **window functions** (LAG, moving average, rankings).
3. Practicar **validación y verificación** de resultados (control totals, sanity checks).
4. Dar **forma de reporte** a salidas SQL (orden, redondeos, filtros ejecutivos).
5. Analizar **tendencias** (por mes, % cambio, rolling).

## Agenda
0) Configuración rápida RunSQL  
1) Reset de esquema y datos extendidos  
2) KPIs con CTEs y CASE  
3) ROI por canal/campaña  
4) Validación y verificación  
5) Reportes ejecutivos  
6) Tendencias con window functions  
7) Ejercicios guiados (+ soluciones)  
8) Retos

## 0) Configuración rápida RunSQL (Breakout Rooms)

Vamos a configurar nuestro entorno en runSQL creando las tablas e insertando los datos. Par esto haremos lo siguiente:
1. Abre **runsql.com** y crea un nuevo playground (gratuito).  
2. Selecciona **PostgreSQL**.  
3. Ejecuta los bloques de la sección 1 en orden.
4. Ejecuta los bloques de la sección 2 en orden.
5. Has un join entre la tabla SALES y PRODUCTS para validar que todo esté funcionando bien.

## 1) Reset y creación de tablas (extendido)
Reusaremos el esquema de la sesión anterior y añadiremos una tabla de canales/día para tendencias.

In [ ]:
-- 1.1 Reset
DROP TABLE IF EXISTS campaign_daily; 
DROP TABLE IF EXISTS marketing_spend; 
DROP TABLE IF EXISTS sales; 
DROP TABLE IF EXISTS products; 
DROP TABLE IF EXISTS stores;

In [ ]:
-- 1.2 stores
CREATE TABLE stores (
  store_id    INTEGER PRIMARY KEY,
  store_name  VARCHAR(80) NOT NULL,
  region      VARCHAR(40) NOT NULL
);

-- 1.3 products
CREATE TABLE products (
  product_id   INTEGER PRIMARY KEY,
  product_name VARCHAR(80) NOT NULL,
  category     VARCHAR(40) NOT NULL,
  unit_cost    NUMERIC(12,2) NOT NULL,
  unit_price   NUMERIC(12,2) NOT NULL
);

-- 1.4 sales
CREATE TABLE sales (
  sale_id    INTEGER PRIMARY KEY,
  sale_date  DATE NOT NULL,
  store_id   INTEGER NOT NULL REFERENCES stores(store_id),
  product_id INTEGER NOT NULL REFERENCES products(product_id),
  units      INTEGER NOT NULL CHECK (units >= 0)
);

-- 1.5 marketing_spend
CREATE TABLE marketing_spend (
  campaign_id   INTEGER PRIMARY KEY,
  campaign_name VARCHAR(80) NOT NULL,
  start_date    DATE NOT NULL,
  end_date      DATE NOT NULL,
  channel       VARCHAR(40) NOT NULL,
  spend         NUMERIC(12,2) NOT NULL CHECK (spend >= 0),
  store_id      INTEGER NOT NULL REFERENCES stores(store_id)
);

-- 1.6 campaign_daily (tracking diario por canal para tendencias)
CREATE TABLE campaign_daily (
  cd_id        SERIAL PRIMARY KEY,
  campaign_id  INTEGER NOT NULL REFERENCES marketing_spend(campaign_id),
  day         DATE NOT NULL,
  channel     VARCHAR(40) NOT NULL,
  impressions INTEGER NOT NULL,
  clicks      INTEGER NOT NULL,
  spend       NUMERIC(12,2) NOT NULL
);

## 2) Inserción de datos (extendido para tendencias)

In [ ]:
-- 2.1 stores
INSERT INTO stores (store_id, store_name, region) VALUES
(1, 'Cali Norte',  'West'),
(2, 'Cali Sur',    'West'),
(3, 'Bogota Centro','East'),
(4, 'Medellin',    'North');

-- 2.2 products
INSERT INTO products (product_id, product_name, category, unit_cost, unit_price) VALUES
(101, 'Mouse',        'Accesorios', 12.00, 20.00),
(102, 'Teclado',      'Accesorios', 18.00, 30.00),
(103, 'Headphones',   'Audio',      35.00, 60.00),
(104, 'Tablet 8"',   'Tablets',    80.00, 120.00);

-- 2.3 sales (Q3-2025 con más filas)
INSERT INTO sales (sale_id, sale_date, store_id, product_id, units) VALUES
(1,'2025-07-03',1,101,20),(2,'2025-07-07',1,102,18),(3,'2025-07-12',1,103,10),(4,'2025-07-17',1,104,6),
(5,'2025-08-02',2,101,22),(6,'2025-08-09',2,104,12),(7,'2025-08-16',2,103,14),(8,'2025-08-23',2,102,15),
(9,'2025-09-01',3,101,30),(10,'2025-09-10',3,104,9),(11,'2025-09-18',3,102,20),(12,'2025-09-27',3,103,12),
(13,'2025-07-05',4,103,8),(14,'2025-07-19',4,102,14),(15,'2025-08-12',4,101,25),(16,'2025-09-13',4,103,16);

-- 2.4 marketing_spend (dos campañas por tienda para solape)
INSERT INTO marketing_spend (campaign_id, campaign_name, start_date, end_date, channel, spend, store_id) VALUES
(201,'BackToSchool','2025-08-01','2025-08-31','Ads',   1200.00,1),
(202,'HeadphonesQ3','2025-07-01','2025-09-30','Social', 800.00,2),
(203,'TabletFlash', '2025-09-01','2025-09-15','Email',  300.00,3),
(204,'Accesorios+', '2025-07-15','2025-08-15','Ads',    600.00,4),
(205,'Q3_Remarketing','2025-08-10','2025-09-25','Email', 500.00,1);

-- 2.5 campaign_daily (tracking simulado)
INSERT INTO campaign_daily (campaign_id, day, channel, impressions, clicks, spend) VALUES
(201,'2025-08-05','Ads',  5000, 300, 120.00),
(201,'2025-08-12','Ads',  5500, 320, 160.00),
(201,'2025-08-19','Ads',  5800, 350, 220.00),
(201,'2025-08-26','Ads',  6000, 360, 300.00),
(205,'2025-08-15','Email', 8000, 400, 110.00),
(205,'2025-09-05','Email', 8200, 420, 140.00),
(205,'2025-09-20','Email', 9000, 480, 150.00),
(202,'2025-07-20','Social',7000, 500, 200.00),
(202,'2025-08-18','Social',7500, 520, 220.00),
(202,'2025-09-15','Social',7800, 540, 240.00),
(203,'2025-09-03','Email', 4000, 300, 80.00),
(203,'2025-09-10','Email', 4200, 320, 90.00),
(203,'2025-09-14','Email', 4300, 330, 100.00),
(204,'2025-07-20','Ads',   3000, 180, 90.00),
(204,'2025-08-05','Ads',   3500, 210, 120.00);

## 2.1) Calculemos el Revenue total y el costo total

Usando lo aprendido en la clase anterior; calculemos la sumatoria del costo total y la sumatoria de las ventas de productos.

**Pasos:**

1. Une todas las tablas, seleccion todas las columnas.
2. Cambia el `*` del `SELECT` por las columnas que quieras traer.
3. Especifica la columna categorica y las columnas para calcular el revenue total y el costo total.
4. No olvides colocar la columna categorica en la función `GROUP BY`
5. Puedes complementar tu query usando `ORDER BY`
6. Revisa los valores obtenidos.

## 2bis) KPIs con CTE y CASE
Calcular revenue/cost/gross_profit/margin_pct por categoría y marcar salud del margen.

In [ ]:
SELECT
  p.category,
  SUM(s.units * p.unit_price) AS revenue,
  SUM(s.units * p.unit_cost)  AS cost,
  SUM(s.units * (p.unit_price - p.unit_cost)) AS gross_profit,
  ROUND(
    100.0 * SUM(s.units * (p.unit_price - p.unit_cost))
    / NULLIF(SUM(s.units * p.unit_price), 0)
  , 2) AS margin_pct,
  CASE
    WHEN SUM(s.units * (p.unit_price - p.unit_cost))
         / NULLIF(SUM(s.units * p.unit_price), 0) >= 0.35 THEN 'Excelente'
    WHEN SUM(s.units * (p.unit_price - p.unit_cost))
         / NULLIF(SUM(s.units * p.unit_price), 0) >= 0.20 THEN 'Aceptable'
    ELSE 'Revisar'
  END AS margin_health
FROM sales s
JOIN products p ON p.product_id = s.product_id
GROUP BY p.category
ORDER BY margin_pct DESC;


## 3) ROI por canal/campaña (con daily tracking)
Atribución simple por ventana + tienda; el gasto usa `campaign_daily` (más granular).

In [ ]:
SELECT
  ms.campaign_name,
  st.store_name AS store,
  (ms.start_date || ' -> ' || ms.end_date) AS window,
  COALESCE(SUM(s.units * p.unit_price), 0) AS revenue,
  COALESCE(sd.spend, 0) AS spend,
  ROUND(
    100.0 * (COALESCE(SUM(s.units * p.unit_price), 0) - COALESCE(sd.spend, 0))
    / NULLIF(COALESCE(sd.spend, 0), 0)
  , 2) AS roi_pct
FROM marketing_spend ms
LEFT JOIN stores st
  ON st.store_id = ms.store_id
LEFT JOIN sales s
  ON s.store_id = ms.store_id
 AND s.sale_date BETWEEN ms.start_date AND ms.end_date
LEFT JOIN products p
  ON p.product_id = s.product_id
LEFT JOIN (
  SELECT campaign_id, SUM(spend) AS spend
  FROM campaign_daily
  GROUP BY campaign_id
) sd
  ON sd.campaign_id = ms.campaign_id
GROUP BY
  ms.campaign_name, st.store_name, ms.start_date, ms.end_date, sd.spend
ORDER BY roi_pct DESC NULLS LAST;


## 4) Validación y verificación de resultados
### 4.1 Control total: revenue por dos rutas debería coincidir
Ruta: sumar `units*price` en `sales`×`products`.

In [ ]:
SELECT SUM(s.units * p.unit_price) AS total_revenue_A
FROM sales s JOIN products p ON p.product_id = s.product_id;

### 4.2 Sanity checks
- `units >= 0`  
- precios/costos positivos  
- fechas dentro de Q3-2025

In [ ]:
SELECT 
  SUM(CASE WHEN units < 0 THEN 1 ELSE 0 END) AS bad_units,
  SUM(CASE WHEN unit_price <= 0 OR unit_cost <= 0 THEN 1 ELSE 0 END) AS bad_prices,
  SUM(CASE WHEN sale_date < '2025-07-01' OR sale_date > '2025-09-30' THEN 1 ELSE 0 END) AS out_of_range
FROM sales s JOIN products p ON p.product_id = s.product_id;

## 5) Salidas con forma de reporte (orden/formatos)
Top categorías por margen, 2 decimales, orden descendente.

In [ ]:
SELECT p.category,
       ROUND(SUM(s.units * p.unit_price),2) AS revenue,
       ROUND(SUM(s.units * p.unit_cost),2)  AS cost,
       ROUND(SUM(s.units * p.unit_price) - SUM(s.units * p.unit_cost),2) AS gross_profit,
       ROUND(100.0 * (SUM(s.units * p.unit_price) - SUM(s.units * p.unit_cost)) / NULLIF(SUM(s.units * p.unit_price),0),2) AS margin_pct
FROM sales s JOIN products p ON p.product_id = s.product_id
GROUP BY p.category
ORDER BY margin_pct DESC, revenue DESC;

## 6) Tendencias con window functions
### 6.1 Revenue mensual
### 6.2 % cambio vs mes anterior (LAG)
### 6.3 Media móvil 3 puntos (AVG OVER ... ROWS 2 PRECEDING)

In [ ]:
-- 6.1 Revenue mensual
WITH rev AS (
  SELECT DATE_TRUNC('month', sale_date) AS month,
         SUM(s.units * p.unit_price) AS revenue
  FROM sales s JOIN products p ON p.product_id = s.product_id
  GROUP BY 1
)
SELECT month, revenue
FROM rev
ORDER BY month;

In [ ]:
-- 6.2 % cambio vs mes anterior
WITH rev AS (
  SELECT DATE_TRUNC('month', sale_date) AS month,
         SUM(s.units * p.unit_price) AS revenue
  FROM sales s JOIN products p ON p.product_id = s.product_id
  GROUP BY 1
)
SELECT month,
       revenue,
       ROUND(100.0 * (revenue - LAG(revenue) OVER (ORDER BY month)) / NULLIF(LAG(revenue) OVER (ORDER BY month),0), 2) AS pct_change
FROM rev
ORDER BY month;

In [ ]:
-- 6.3 Media móvil de 3 puntos
WITH daily AS (
  SELECT sale_date::date AS day,
         SUM(s.units * p.unit_price) AS revenue
  FROM sales s JOIN products p ON p.product_id = s.product_id
  GROUP BY 1
)
SELECT day,
       revenue,
       ROUND(AVG(revenue) OVER (ORDER BY day ROWS BETWEEN 2 PRECEDING AND CURRENT ROW), 2) AS ma3
FROM daily
ORDER BY day;

## 7) Ejercicios guiados (RunSQL)
Resuelve y luego compara con la solución.

### Ejercicio 7.1 — Margen por tienda con semáforo (CASE)
Enunciado: calcula revenue, cost, margin_pct por tienda y agrega un semáforo: 'verde' ≥ 25%, 'amarillo' 15–25%, 'rojo' < 15%.

In [ ]:
-- Tu consulta aquí

**Solución propuesta:**

In [ ]:
SELECT st.store_name,
       SUM(s.units * p.unit_price) AS revenue,
       SUM(s.units * p.unit_cost)  AS cost,
       ROUND(100.0 * (SUM(s.units * p.unit_price) - SUM(s.units * p.unit_cost)) / NULLIF(SUM(s.units * p.unit_price),0), 2) AS margin_pct,
       CASE WHEN (SUM(s.units * p.unit_price) - SUM(s.units * p.unit_cost))/NULLIF(SUM(s.units * p.unit_price),0) >= 0.25 THEN 'verde'
            WHEN (SUM(s.units * p.unit_price) - SUM(s.units * p.unit_cost))/NULLIF(SUM(s.units * p.unit_price),0) >= 0.15 THEN 'amarillo'
            ELSE 'rojo' END AS semaforo
FROM sales s JOIN products p ON p.product_id = s.product_id
JOIN stores st ON st.store_id = s.store_id
GROUP BY st.store_name
ORDER BY margin_pct DESC;

### Ejercicio 7.2 — ROI por canal (agregado) y top canal
Enunciado: agrega spend de `campaign_daily` por canal y calcula revenue atribuido aproximado por canal (usando ventanas de cada campaña). ¿Cuál es el mejor canal por ROI?

In [ ]:
-- Tu consulta aquí

**Solución propuesta:**

In [ ]:
SELECT
  r.channel,
  COALESCE(r.revenue, 0) AS revenue,
  COALESCE(s.spend, 0)   AS spend,
  ROUND(
    100.0 * (COALESCE(r.revenue,0) - COALESCE(s.spend,0))
    / NULLIF(COALESCE(s.spend,0), 0)
  , 2) AS roi_pct
FROM (
  SELECT
    ms.channel,
    SUM(s.units * p.unit_price) AS revenue
  FROM marketing_spend ms
  LEFT JOIN sales s
    ON s.store_id = ms.store_id
   AND s.sale_date BETWEEN ms.start_date AND ms.end_date
  LEFT JOIN products p
    ON p.product_id = s.product_id
  GROUP BY ms.channel
) r
LEFT JOIN (
  SELECT channel, SUM(spend) AS spend
  FROM campaign_daily
  GROUP BY channel
) s
USING (channel)
ORDER BY roi_pct DESC NULLS LAST;


### Ejercicio 7.3 — % cambio mensual y detección de caída
Enunciado: calcula revenue mensual y el % cambio vs mes anterior; marca con CASE cuando la caída sea peor que -15%.

In [ ]:
-- Tu consulta aquí

**Solución propuesta:**

In [ ]:
WITH rev AS (
  SELECT DATE_TRUNC('month', sale_date) AS month,
         SUM(s.units * p.unit_price) AS revenue
  FROM sales s JOIN products p ON p.product_id = s.product_id
  GROUP BY 1
)
SELECT month, revenue,
       ROUND(100.0 * (revenue - LAG(revenue) OVER (ORDER BY month)) / NULLIF(LAG(revenue) OVER (ORDER BY month),0),2) AS pct_change,
       CASE WHEN (revenue - LAG(revenue) OVER (ORDER BY month)) / NULLIF(LAG(revenue) OVER (ORDER BY month),0) < -0.15 THEN 'alerta'
            ELSE 'ok' END AS flag
FROM rev
ORDER BY month;

## 8) Retos (opcional)
1) **Cohortes por tienda**: primera compra del trimestre y revenue acumulado por tienda (usar MIN(sale_date) OVER PARTITION y SUM OVER).  
2) **Basket mix simple**: por tienda, top 1 categoría por revenue/costo y % participación.  
3) **Atribución alternativa**: reparte revenue de ventas en ventanas solapadas proporcionalmente al gasto diario de cada campaña (pista: unir sales con campaign_daily por fecha y tienda; normaliza gasto por día).

---

**Generado:** 2025-10-20 22:24 · Listo para RunSQL (PostgreSQL).

## Cierre
**Kahoot de repaso (5 min)**
- Practicamos JOINs para cruzar tablas de ventas y productos.
- Calculamos ROI por campaña/canal.

**Reflexión:**
- ¿Qué pasa si hacemos un JOIN incorrecto (duplicidad de datos)?
- ¿Cómo validaste que tus números fueran coherentes hoy?

**Q&A y próximos pasos.**


## Siguientes Pasos
- **Próxima sesión:** Sprint 4 - Funnels y User Journey.
- **Participación:** Asiste al Sprint Focus para resolver dudas de SQL avanzado.
- **Recordatorios:** Guarda tus scripts de SQL para usarlos como plantilla.
